## Create BoundingBoxes for multiple Point-Locations

Create a bounding box around a number of point locations. This can be useful if you want to sample the area surrounding known point locations - e.g. for collecting training data.

Calculate the bounding box. Code is taken from a [StackOverflow answer by Frederico A. Ramponi](http://stackoverflow.com/a/238558/4169585).

In [1]:
import math

# degrees to radians
def deg2rad(degrees):
    return math.pi*degrees/180.0
# radians to degrees
def rad2deg(radians):
    return 180.0*radians/math.pi

# Semi-axes of WGS-84 geoidal reference
WGS84_a = 6378137.0  # Major semiaxis [m]
WGS84_b = 6356752.3  # Minor semiaxis [m]

# Earth radius at a given latitude, according to the WGS-84 ellipsoid [m]
def WGS84EarthRadius(lat):
    # http://en.wikipedia.org/wiki/Earth_radius
    An = WGS84_a*WGS84_a * math.cos(lat)
    Bn = WGS84_b*WGS84_b * math.sin(lat)
    Ad = WGS84_a * math.cos(lat)
    Bd = WGS84_b * math.sin(lat)
    return math.sqrt( (An*An + Bn*Bn)/(Ad*Ad + Bd*Bd) )

# Bounding box surrounding the point at given coordinates,
# assuming local approximation of Earth surface as a sphere
# of radius given by WGS84
def boundingBox(latitudeInDegrees, longitudeInDegrees, halfSideInKm):
    lat = deg2rad(latitudeInDegrees)
    lon = deg2rad(longitudeInDegrees)
    halfSide = 1000*halfSideInKm

    # Radius of Earth at given latitude
    radius = WGS84EarthRadius(lat)
    # Radius of the parallel at given latitude
    pradius = radius*math.cos(lat)

    latMin = lat - halfSide/radius
    latMax = lat + halfSide/radius
    lonMin = lon - halfSide/radius
    lonMax = lon + halfSide/radius

    x=(rad2deg(latMin), rad2deg(lonMin), rad2deg(latMax), rad2deg(lonMax))
    return x

Given a GeoJSON input file with multiple point locations calculate a output GeoJSON files with rectangular polygons whose center is at the point locations.

In [30]:
input_file = "PRD_20randompoints.geojson"
output_file = "PRD_20randomBB.geojson"
halfside = 0.2  # half side of bounding Box in kilometres

In [31]:
import geojson

with open(input_file, 'r') as infile:
     points=geojson.load(infile)

poly_list = []  # list for new polygon features

# loop through point list and create bB Polygons
for fid, feat in enumerate(points["features"]):
        point_cord = feat["geometry"]["coordinates"]
        bB = boundingBox(point_cord[0], point_cord[1], halfside)  # create boundingBox around point
        # create bB Polygons and append Features to result list
        poly_cord = geojson.Polygon([[[bB[0], bB[1]], [bB[0], bB[3]], [bB[2], bB[3]], [bB[2], bB[1]], [bB[0], bB[1]]]])
        poly_list.append(geojson.Feature(geometry=poly_cord, id=fid, properties={}))
        
# write to disk, convert list of Features to FeatureCollection
with open(output_file, "w") as outfile:
    geojson.dump(geojson.FeatureCollection(poly_list), outfile)